In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier


In [2]:
df_with_emojis = pd.read_csv('../datasets/df_with_emojis.csv')
df_without_emojis = pd.read_csv('../datasets/df_without_emojis.csv')

df_with_emojis = df_with_emojis.dropna()
df_without_emojis = df_without_emojis.dropna()

In [3]:
X_train_emojis, X_test_emojis, y_train_emojis, y_test_emojis = train_test_split(
    df_with_emojis['text'], df_with_emojis['sentiment'], test_size=0.2, random_state=42)

X_train_no_emojis, X_test_no_emojis, y_train_no_emojis, y_test_no_emojis = train_test_split(
    df_without_emojis['text'], df_without_emojis['sentiment'], test_size=0.2, random_state=42)

In [4]:
vectorizer = TfidfVectorizer(max_features=5000) 
X_train_emojis_vec = vectorizer.fit_transform(X_train_emojis)
X_test_emojis_vec = vectorizer.transform(X_test_emojis)
X_train_no_emojis_vec = vectorizer.fit_transform(X_train_no_emojis)
X_test_no_emojis_vec = vectorizer.transform(X_test_no_emojis)

print(f"Formato da matriz TF-IDF (treino) - com emojis: {X_train_emojis_vec.shape}")
print(f"Formato da matriz TF-IDF (treino) - sem emojis: {X_train_no_emojis_vec.shape}")

Formato da matriz TF-IDF (treino) - com emojis: (1274444, 5000)
Formato da matriz TF-IDF (treino) - sem emojis: (1274420, 5000)


In [5]:
def plot_confusion_matrix(y_true, y_pred, title, class_labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.title(title)
    plt.ylabel('Verdadeiro')
    plt.xlabel('Previsto')
    plt.show()

In [ ]:
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_emojis_vec)
X_test_scaled = scaler.transform(X_test_emojis_vec)

# Treinando o modelo MLP
mlp_emojis = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', batch_size=32, max_iter=200, random_state=42, verbose=True)
mlp_emojis.fit(X_train_scaled, y_train_emojis)

# Avaliação
y_pred_emojis = mlp_emojis.predict(X_test_scaled)
print("Relatório de classificação:")
print(classification_report(y_test_emojis, y_pred_emojis))


In [ ]:
class_labels = np.unique(y_test_emojis)

plot_confusion_matrix(y_test_emojis, y_pred_emojis, "Matriz de Confusão - MLP com emojis ", class_labels)

In [ ]:
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_no_emojis_vec)
X_test_scaled = scaler.transform(X_test_no_emojis_vec)

# Treinando o modelo MLP
mlp_no_emojis = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', batch_size=32, max_iter=200, random_state=42, verbose=True)
mlp_no_emojis.fit(X_train_scaled, y_train_no_emojis)

# Avaliação
y_pred_no_emojis = mlp_emojis.predict(X_test_scaled)
print("Relatório de classificação:")
print(classification_report(y_test_no_emojis, y_pred_no_emojis))


In [ ]:
class_labels = np.unique(y_test_emojis)

plot_confusion_matrix(y_test_emojis, y_pred_emojis, "Matriz de Confusão - MLP sem emojis ", class_labels)